## Test Code for the cGAN

In [8]:
import torch
import pathlib
def celeb_label_transform(desired_attr):
    """
    Pass in a the names of all the attributes that you want
    """

    file = open('C:/Users/Paddy/CRT/Github/input/CelebA/list_attr_celeba.csv').read().split()
    attr_names = file[0].split(',')
    file = file[1:]
    
    def transform(idx):
        attr = torch.tensor([int(entry) for entry in file[idx].split(',')[1:]])
        mask = [attr_names[1:][i] in desired_attr for i in range(len(attr))]
        masked = attr[mask]
        return torch.relu(masked).float()
    return transform

file = open('C:/Users/Paddy/CRT/Github/input/CelebA/list_attr_celeba.csv').read().split()
attr_names = file[0].split(',')

bald_transform = celeb_label_transform("Bald")
bald_transform(50)


tensor([1.])

In [9]:
import pandas as pd

targ_dir = 'C:/Users/Paddy/CRT/Github/input/SEN12MS/'
filenames = list(pathlib.Path(targ_dir).glob("*/*/*.tif"))

# dictionary of lists
dict = {'name': filenames}
     
df = pd.DataFrame(filenames)
     
# saving the dataframe
df.to_csv('sen12_filenames.csv')

In [6]:
def sen12_label_transform(desired_season):
    """
    Pass in a the names of all the attributes that you want
    """

    file = open('C:/Users/Paddy/CRT/Github/input/SEN12MS/sen12_seasons.csv').read().split()
    season_names = file[0].split(',')
    file = file[1:]
    
    def transform(idx):
        season = torch.tensor([int(entry) for entry in file[idx].split(',')[1:]])
        mask = [season_names[1:][i] in desired_season for i in range(len(season))]
        masked = season[mask]
        return torch.relu(masked).float()
    return transform

C:\Users\Paddy\CRT\Github\input\SEN12MS\ROIs1158_spring\s2_1\ROIs1158_spring_s2_1_p100.tif


In [27]:
import os

path = "C:/Users/Paddy/CRT/Github/input/SEN12MS/"
filenames = list(pathlib.Path(targ_dir).glob("*/*/*.tif"))

path = "C:/Users/Paddy/CRT/Github/input/SEN12MS/ROIs1158_spring/s2_1/ROIs1158_spring_s2_1_p100.tif"
if path.split(os.path.sep)[-1]:
    parent_folder = path.split(os.path.sep)[-1] # if no backslashes at the end
else:
    parent_folder = path.split(os.path.sep)[-2]

print(parent_folder)

C:/Users/Paddy/CRT/Github/input/SEN12MS/ROIs1158_spring/s2_1/ROIs1158_spring_s2_1_p100.tif
